# Face Recognition w/ 3D Landmarks

Melisa Mete, 150200316

Öykü Eren, 150200326

Bora Boyacıoğlu, 150200310

## Step 2: Model Implementation and Training

Import necessary libraries.

In [ ]:
from datetime import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import sys

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim

In [ ]:
%reload_ext autoreload
%autoreload 2

Load Torch device.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


### 2.1. Open the Preprocessed Data

In [ ]:
# Change this if running on drive.
on_drive = True
home_dir = ''
model_dir = ''

if on_drive:
    home_dir = '/content/drive/MyDrive/CVProject/'
    model_dir = home_dir + 'model/'
    sys.path.append(home_dir + 'model/utils')

data_dir = home_dir + data_dir

from utils.model import GhostFaceNetsV2
from utils.face_dataset import FaceDataset

In [ ]:
# Data paths.
train_paths= [data_dir + 'X1_train.npy',
              data_dir + 'X2_train.npy',
              data_dir + 'y_train.npy']

test_paths = [data_dir + 'X1_test.npy',
              data_dir + 'X2_test.npy',
              data_dir + 'y_test.npy']

# Create the datasets.
train_dataset = FaceDataset(*train_paths)
test_dataset = FaceDataset(*test_paths)

# Create data loaders.
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

### 2.2. Initialise the Model

In [ ]:
# Define the model parameters.
size = train_dataset.size()
length = len(train_dataset)
width = 1
dropout = 0.3

# Model type: Combined, Landmark or Image.
combined = 'Combined'

# Other parameters.
lr = 0.0001

In [ ]:
# Create the model.
model = GhostFaceNetsV2(image_size=size, num_classes=length, width=width, dropout=dropout, combined=combined)

model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr)

### 2.3. Training Loop

In [ ]:
# Training loop parameters.
num_epochs = 10
epoch = 0
timestamp = dt.now().strftime('%d_%H_%M')
!mkdir -p {model_dir}model/{timestamp}

In [ ]:
print(f"\033[92mTraining started from epoch {epoch + 1} to {num_epochs}.\033[0m")

while epoch < num_epochs:
    print(f"\n\033[93mEpoch {epoch + 1}/{num_epochs}\033[0m")

    """ TRAINING STEP """
    # Set the model to training mode.
    model.train()
    running_loss = 0.0

    for i, (images, landmarks, labels) in enumerate(train_loader):
        print(f"\rBatch {i + 1}/{len(train_loader)} ({100 * (i + 1) / len(train_loader):.2f}%), Epoch Loss: {running_loss / len(train_loader):.4f}", end='')

        images, landmarks, labels = images.to(device), landmarks.to(device), labels.to(device)

        optimizer.zero_grad()
        if model.combined == 'Image':
            outputs = model(x=images)
        elif model.combined == 'Landmark':
            outputs = model(landmarks=landmarks)
        elif model.combined == 'Combined':
            outputs = model(x=images, landmarks=landmarks)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print()

    """ VALIDATION STEP """
    # Set the model to evaluation mode.
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for i, (images, landmarks, labels) in enumerate(test_loader):
            print(f"\rValidation Batch {i + 1}/{len(test_loader)} ({100 * (i + 1) / len(test_loader):.2f}%)", end='')

            images, landmarks, labels = images.to(device), landmarks.to(device), labels.to(device)
            if model.combined == 'Image':
                outputs = model(x=images)
            elif model.combined == 'Landmark':
                outputs = model(landmarks=landmarks)
            elif model.combined == 'Combined':
                outputs = model(x=images, landmarks=landmarks)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f", Validation Accuracy: {correct / total:.4f}")

    epoch += 1

    # Save the trained model for the epoch.
    torch.save(model.state_dict(), f"{model_dir}model/{timestamp}/epoch_{epoch}.pt")
